In [2]:
import cv2 as cv
import numpy as np
from pathlib import Path

## Save the images from the entry camera into a database

In [3]:
# image database (contains images taken by the entry camera)
db = []

folder_dir = 'base_photos'    
images = Path(folder_dir).glob('*.jpg')

for image in images:
    tmp = str(image)[len(folder_dir)+1:][:-4].split("-")
    shoe_id = int(tmp[0])
    num = int(tmp[1])
    img = cv.imread(str(image))
    db.append({'image':img, 'id':shoe_id, 'number':num})
    
# to display the 6th picture in the photos directory
# cv.imshow('test', db[5]['image'])
# cv.waitKey(0)

So far, I only saved the original pictures into the db. I haven't done image augmentation (eg. flip, rotate) on them. Moving on, we should add the augmented images into the db, with the same `id` but with a diff `number`

Also, the names of the images in `base_photos` is saved such that the number before the - is the id number, while the number after the - is just to show which photo of the id is this. For example, `2-3.jpg` means this shoe is id 2 and this photo is the 3rd photo of id 2.

## Testing the Template Matching method

We want to find the ids of shoes in folder `to_retrieve_ids`. Note that in the folder, only `1.jpg` is exactly the same as picture `4-1.jpg` in `base_photos`. Other pictures in `to_retrieve_ids` are not exact replicate of pictures in `base_photos`.

In [56]:
folder_dir = 'to_retrieve_ids'
images = Path(folder_dir).glob('*.jpg')

actual_ids = [4, 4, 2, 3]
identified = []    # stores result from template matching

for image in images:
    template = cv.imread(str(image), cv.IMREAD_COLOR)
    threshold = 0.7     # set threshold; high (ie. 1) means "more strict"
    tmp = []    # stores detected ids for this shoe; if empty means no id is assigned to this shoe
    
    # perform match operation
    for item in db:
        res = cv.matchTemplate(item['image'], template, cv.TM_CCOEFF_NORMED)
        loc = np.where(res >= threshold)
        if (len(loc[0])!=0 and len(loc[1])!=0):    # if there's a match
            tmp.append(item['id'])
            
    identified.append(tmp)
    
identified

[[2, 4, 4], [1, 2, 4, 4, 4], [1, 2, 4, 4], []]

Here's an interpretation of the variable `identified`:

For this threshold, `1.jpg` in `to_retrieve_ids` is correctly assigned to id 4, but is mistaken to be id 2 as well. Same for `2.jpg` which is mistaken to be id 1 and id 2. The last element of `identified` is an empty list, which means that the algorithm failed to identify that `4.jpg` is actually id 3.

Note: after changing the threshold to 0.85, ids are correctly assigned to `1.jpg` and `2.jpg`. But the algorithm failed to recognise that `3.jpg` is actually id 2 and `4.jpg` is id 3.

We shall see if it will improve with image augmentation + background removal

In [49]:
# to visualize bounding boxes
a = cv.imread('base_photos/4-2.jpg')     # can change
b = cv.imread('to_retrieve_ids/1.jpg', cv.IMREAD_COLOR)    # can change

res = cv.matchTemplate(a, b, cv.TM_CCOEFF_NORMED)
w, h = b.shape[1], b.shape[0]
threshold = 0.7
loc = np.where(res >= threshold)
for pt in zip(*loc[::-1]):
    cv.rectangle(a, pt, (pt[0] + w, pt[1] + h), (0, 255, 255), 2)
cv.imshow('Detected', a)
cv.waitKey(0)

-1